<a href="https://colab.research.google.com/github/shivaprajapati34390-netizen/ML-project/blob/main/stock_price_prediction_with_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import yfinance as yf
import datetime
from datetime import date,timedelta
today=date.today()

In [6]:
d1=today.strftime("%Y-%m-%d")
end_date=d1
d2=date.today()-timedelta(days=5000)
d2=d2.strftime("%Y-%m-%d")

data=yf.download('AAPL',start=d2,end=end_date,progress=False)
data["Date"]=data.index
data=data[["Date","Open","High","Low","Close","Volume"]]
data.reset_index(drop=True,inplace=True)
print(data.tail())

Price        Date        Open        High         Low       Close    Volume
Ticker                   AAPL        AAPL        AAPL        AAPL      AAPL
3435   2026-02-02  260.029999  270.489990  259.209991  270.010010  73913400
3436   2026-02-03  269.200012  271.880005  267.609985  269.480011  64394700
3437   2026-02-04  272.290009  278.950012  272.290009  276.489990  90545700
3438   2026-02-05  278.130005  279.500000  273.230011  275.910004  52977400
3439   2026-02-06  277.119995  280.910004  276.929993  278.119995  50420700


/tmp/ipython-input-3766217103.py:6: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data=yf.download('AAPL',start=d2,end=end_date,progress=False)
